In [ ]:
class UnitStep
  def initialize(delay)
    @delay = delay
  end
  def at(t)
    if t - @delay > 0
      1
    else
      0
    end
  end
end

class FixedFilter
  def initialize(window)
    @window = window
    @data = []
  end
  
  def work(u)
    @data.push u
    if @data.size > @window
      @data.shift
    end
    
    @data.inject(0.0) { |sum, i| sum += i } / @data.size
  end
end

In [ ]:
class Cache
  def initialize(size, demand)
    @t = 0
    
    @size = size
    @cache = {}
    
    @demand = demand
  end
  
  def work(u)
    @t += 1
    
    @size = [0, u.floor].max
    
    i = @demand.at(@t)
    
    if @cache.has_key? i
      @cache[i] = @t
      return 1
    end
    if @cache.size >= @size
      m = 1 + @cache.size - @size
      tmp = {}
      @cache.each { |k, v| tmp[v] = k}
      tmp.keys.sort.each do |k|
        @cache.delete tmp[k]
        m -= 1
        break if m == 0
      end
    end
    
    @cache[i] = @t
    0
  end
end

class SmoothCache
  def initialize(size, demand, window)
    @cache = Cache.new size, demand
    @filter = FixedFilter.new window
  end
  
  def work(u)
    y = @cache.work u
    @filter.work y
  end
end

In [ ]:
require 'gsl'

class CharacterizationDemand
  def initialize(seed = 0)
    @rng = GSL::Rng.alloc 'gsl_rng_mt19937', seed
  end
  def at(t)
    (@rng.gaussian 15).floor
  end
end

class CharacterizationReference
  def at(t)
    0.7 # 70% Cache utilization target
  end
end

# Process Characteristic

Get steady-state response for various cache sizes at workload `CharacterizationDemand`

In [ ]:
plants = 5.times.map do |i|
  demand = CharacterizationDemand.new i
  SmoothCache.new 0, demand, 100
end

result = []
1.upto(120) do |u|
  0.upto(350) do |t|
    result << [u, plants[2].work(u)]
  end
end

require 'gnuplot'
Gnuplot.open do |gp|
  Gnuplot::Plot.new gp do |plot|
    plot.terminal 'png'
    plot.output 'char.png'
    plot.xlabel 'Cache Size'
    plot.ylabel 'Cache Hit Rate'
    plot.data << Gnuplot::DataSet.new(result.transpose) do |ds|
      ds.title = 'Demand B'
    end
  end
end

File.open 'char.png'

In [ ]:
class Demand
  def initialize
    @rng = GSL::Rng.alloc
  end

  def at(t)
    if t < 3000
      @rng.gaussian(15).floor
    elsif t < 5000
      @rng.gaussian(35).floor
    else
      100 + @rng.gaussian(15).floor
    end
  end
end
demand = Demand.new
reference = CharacterizationReference.new
plant = SmoothCache.new 0, demand, 100

class PidController
  def initialize(kp, ki, kd=0.0)
    @kp, @ki, @kd = [kp, ki, kd]
    
    @i = 0
    @d = 0
    @prev = 0
  end
  
  def work(e)
    @i += e
    @d = (e - @prev) 
    @prev = e
    
    return @kp * e + @ki * @i + @kd * @d
  end
end

controller = PidController.new 100, 4.3

y = 0
result = 0.upto(10000).map do |t|
  r = reference.at(t)
  e = r - y
  u = controller.work(e)
  y = plant.work(u)
  [t, u, e, y]
end.transpose

Gnuplot::Plot.new do |plot|
  plot.y2tics
  plot.ylabel 'Cache Hit Rate'
  plot.y2label 'Cache Size'
  plot.data << Gnuplot::DataSet.new(result.values_at 0, 3) do |ds|
    ds.title = 'Cache Hit Rate'
    ds.with = 'lines'
  end
  plot.data << Gnuplot::DataSet.new(result.values_at 0, 1) do |ds|
    ds.title = 'Cache Size'
    ds.with = 'lines'
    ds.axes = 'x1y2'
  end
end

In [ ]:
class TargetUtilization
  def at(t)
    if(t< 3000)
      0.80
    else
      0.40
    end
  end
end

class ServerLoad
  def initialize
    @rng = GSL::Rng.alloc
  end

  def at(t)
    if t < 6000
      @rng.gaussian(50).floor
    else
      @rng.gaussian(200).floor
    end
  end
end

disturbance = ServerLoad.new
reference = TargetUtilization.new
plant = SmoothCache.new 0, disturbance, 100

controller = PidController.new 100, 4.3

y = 0
result = 0.upto(10000).map do |t|
  r = reference.at(t)
  e = r - y
  u = controller.work(e)
  y = plant.work(u)
  [t, u, e, y]
end.transpose

Gnuplot.open do |gp|
Gnuplot::Plot.new(gp) do |plot|
  plot.y2tics
  plot.ylabel 'CPU Utilization'
  plot.y2label 'Number of Instances'
  plot.xlabel 'Time (s)'
    
  plot.terminal 'pdf'
  plot.output 'scaling.pdf'
  plot.data << Gnuplot::DataSet.new(result.values_at 0, 3) do |ds|
    ds.title = 'CPU Utilization'
    ds.with = 'lines'
  end
  plot.data << Gnuplot::DataSet.new(result.values_at 0, 1) do |ds|
    ds.title = 'Number of Instances'
    ds.with = 'lines'
    ds.axes = 'x1y2'
  end
end
end